# Learn how to read GPIO

Start from examples/peripherals/bsp

Some hints here: https://devzone.nordicsemi.com/f/nordic-q-a/10020/struggling-with-basic-gpio-read-on-nrf52/37132#37132
And here: https://devzone.nordicsemi.com/f/nordic-q-a/25570/code-works-with-nothing-in-sdk_config-h-enabled/100804#100804

This version of bsp logs switch state with pin p0.8 pulled low (10k)



```
#include <stdbool.h>
#include <stdint.h>
#include "boards.h"
#include "bsp.h"
#include "app_timer.h"
#include "nordic_common.h"
#include "nrf_error.h"


#include "nrf_log.h"
#include "nrf_log_ctrl.h"
#include "nrf_log_default_backends.h"

#define BUTTON_PREV_ID           0                           /**< Button used to switch the state. */
#define BUTTON_NEXT_ID           1                           /**< Button used to switch the state. */
#define SWITCH_0                 8

static bsp_indication_t actual_state =  BSP_INDICATE_FIRST;         /**< Currently indicated state. */

static const char * indications_list[] = BSP_INDICATIONS_LIST;

/**@brief Function for handling bsp events.
 */
void bsp_evt_handler(bsp_event_t evt)
{
    uint32_t err_code;
    switch (evt)
    {
        case BSP_EVENT_KEY_0:
            if (actual_state != BSP_INDICATE_FIRST)
                actual_state--;
            else
                actual_state = BSP_INDICATE_LAST;
            break;

        case BSP_EVENT_KEY_1:

            if (actual_state != BSP_INDICATE_LAST)
                actual_state++;
            else
                actual_state = BSP_INDICATE_FIRST;
            break;

        default:
            return; // no implementation needed
    }
    err_code = bsp_indication_set(actual_state);
    NRF_LOG_INFO("%s", (uint32_t)indications_list[actual_state]);
    APP_ERROR_CHECK(err_code);
}


/**@brief Function for initializing low frequency clock.
 */
void clock_initialization()
{
    NRF_CLOCK->LFCLKSRC            = (CLOCK_LFCLKSRC_SRC_Xtal << CLOCK_LFCLKSRC_SRC_Pos);
    NRF_CLOCK->EVENTS_LFCLKSTARTED = 0;
    NRF_CLOCK->TASKS_LFCLKSTART    = 1;

    while (NRF_CLOCK->EVENTS_LFCLKSTARTED == 0)
    {
        // Do nothing.
    }
}


/**@brief Function for initializing bsp module.
 */
void bsp_configuration()
{
    uint32_t err_code;

    err_code = bsp_init(BSP_INIT_LEDS | BSP_INIT_BUTTONS, bsp_evt_handler);
    APP_ERROR_CHECK(err_code);

    nrf_gpio_cfg_sense_input(SWITCH_0, NRF_GPIO_PIN_PULLDOWN, NRF_GPIO_PIN_SENSE_HIGH);
}


/**
 * @brief Function for application main entry.
 */
int main(void)
{
    clock_initialization();

    uint32_t err_code = app_timer_init();
    APP_ERROR_CHECK(err_code);

    APP_ERROR_CHECK(NRF_LOG_INIT(NULL));
    NRF_LOG_DEFAULT_BACKENDS_INIT();

    NRF_LOG_INFO("BSP example started.");
    bsp_configuration();

    uint32_t last = 0;

    while (true)
    {
        NRF_LOG_FLUSH();
        __SEV();
        __WFE();
        __WFE();
        // no implementation needed
	
	uint32_t switch0 = nrf_gpio_pin_read(SWITCH_0);
	if (switch0) {
	  if (!last) {
		  NRF_LOG_INFO("switch ON");
	  }
	} else {
		if (last) {
			NRF_LOG_INFO("switch OFF");
		}
	}
	last = switch0;
    }
}


/** @} */
```

## Switch to this notebook for nRF52840 groks
2018-9-11

Once again, lower my expectations.  To have something to demo, Thursday, for the hackathon ...

* Forget the board side, focus on the sensors.  
* Program the 52840 enough to read the sensors and display their value via NRF_LOG.
* Using polling
* Separate demo program for each sensor

### Tilt Switch

* Start from BSP
* Copy the template sdk_config.h to give me a better chance at having all the flags I need later.
